In [ ]:
# Importing required libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3 # libraries for database file sqlite
import sqlalchemy
from sqlalchemy import create_engine


URL = "https://www.journaldunet.com/business/salaire/index/departements"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

result = soup.find_all("ul", class_ = "bloc size1of2 odSquareList" )

# print(result)

hrefs = []

# The href attribute indicates the destination of the hyperlink
for x in result:
    hrefs += x.find_all("a", href = True)
    

province = []
france_affluence = []



for link in hrefs:
    url = link['href']

    # print(url)
    
    # Retreving the name directly from the link
    province_name = url.split('/')[-2]
    
    # print(province_name)

    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id="jStickySize")
    results2 = results.find(class_="hidden marB20")
    
    
    # ['2\xa0285 € nets', '27\xa0425 € nets']
    # <class 'list'>
    header = [b.string for h in results2.find_all('p') for b in h.find_all('b')][1]
    
    
    # print(header)
    # print(type(header))

 

    text = header.replace(u'\xa0', u' ')
    text_2 = text.replace(" ", "")
    text_3 = text_2.strip('€nets')



 
    province.append(province_name)
    france_affluence.append(text_3)

    

    aff_dict = {
        "province" : province,
        "france_affluence": france_affluence
    }


    
    affluence_df = pd.DataFrame(aff_dict, columns=["province", "france_affluence"])

    
    #aff_df.to_csv('france_affluence', sep='\t', index=False)
    
    print(affluence_df)

    
    DATABASE_LOCATION = "sqlite:///raw_hyperlocal_affluence_fr.sqlite"
    engine = sqlalchemy.create_engine(DATABASE_LOCATION)
    conn = sqlite3.connect('raw_hyperlocal_affluence_fr.sqlite')
    cursor = conn.cursor()

    sql_query = """
    CREATE TABLE IF NOT EXISTS raw_hyperlocal_affluence_fr(
                            province VARCHAR(200),
                            affluence_france VARCHAR(200)
    )
    """
    
    cursor.execute(sql_query)

    
    try:
        affluence_df.to_sql("raw_hyperlocal_affluence_fr", engine, if_exists='replace', index=False)
    except:
        print("Data already exists in the database")
   
    
    conn.close()
    print("Close database successfully")



    # DBeaver is a free database managment software